# Charpter 3 Data classification forecast  数据分类与预测

### 实验目的

使用scikit-learn 包中的tree，贝叶斯，knn，对数据进行模型训练，尽量了解其原理及运用。

使用不同分析三种分类器在实验中的性能比较，分析它们的特点

### 实验介绍：

在挖掘前先分析与预处理数据

**sklearn 简介**

Scikit-learn(sklearn)是机器学习中常用的第三方模块，
对常用的机器学习方法进行了封装，包括回归(Regression)、
降维(Dimensionality Reduction)、分类(Classfication)、
聚类(Clustering)等方法。
当我们面临机器学习问题时，便可根据下图来选择相应的方法。

Sklearn具有以下特点：

    简单高效的数据挖掘和数据分析工具
    让每个人能够在复杂环境中重复使用
    建立NumPy、Scipy、MatPlotLib之上



##### 数据集

本实验采用的数据集为house与segment,放在data文件夹下。

导入必要的包

In [206]:
from sklearn import tree
import pandas as pd
import numpy as np

观察一下对应的数据集A ：
house-votes-84.data 是1984年美国国会投票记录数据库
相关信息：

该数据集包括美国国会众议员对CQA确定的16张关键选票的投票。
CQA列出了九种不同类型的投票：投票赞成、配对赞成和宣布赞成
（这三种简化为“是”）、
投票反对、配对反对和宣布反对（这三种简化为“否”）、
投票出席、投票出席以避免利益冲突，并且没有投票或以其他方式表明立场
（这三种情况简化为未知的处置方式？）

观察一下文件格式：
![](Pic/vote.JPG)

这里换行的逗号隔开，考虑使用当成csv格式文件用pandas库进行处理

In [207]:
vote_data = pd.read_csv('data/house-votes-84.data',header=None) # 获取日期数据
vote_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,n,n,y,y,y,y,n,n,y,y,n,y,y,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
432,republican,n,?,n,y,y,y,n,n,n,n,y,y,y,y,n,y
433,republican,n,n,n,y,y,y,?,?,?,?,n,y,y,y,n,y


### 数据预处理

下面我们需要对数据进行预处理操作。

处理时根据说明，发现问题:

注意：必须认识到“？”在这个数据库里并不意味着属性的值是未知的。它
简单地说，这个值不是“是”或“不是”（参见“相关信息”部分）。

**如何解决？**

这里暂时把这些y/n/?用数值模式替换，而不是布尔值，将y表示1，n表示-1，？b表示为0进行替代

判断一下是否有缺失值

In [208]:
vote_data.isnull().any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool

In [209]:
vote_data = vote_data.replace('y',1).replace('n',-1).replace('?',0)
vote_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,-1,1,-1,1,1,1,-1,-1,-1,1,0,1,1,1,-1,1
1,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,0
2,democrat,0,1,1,0,1,1,-1,-1,-1,-1,1,-1,1,1,-1,-1
3,democrat,-1,1,1,-1,0,1,-1,-1,-1,-1,1,-1,1,-1,-1,1
4,democrat,1,1,1,-1,1,1,-1,-1,-1,-1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,republican,-1,-1,1,1,1,1,-1,-1,1,1,-1,1,1,1,-1,1
431,democrat,-1,-1,1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,-1,1
432,republican,-1,0,-1,1,1,1,-1,-1,-1,-1,1,1,1,1,-1,1
433,republican,-1,-1,-1,1,1,1,0,0,0,0,-1,1,1,1,-1,1


可以看到对应的y/n/? 都被数字替代了！

此外，第一列的数据不是以数字或者布尔格式形成，容易导致出现异常报错，
因此我们也约定，republican用1替代，democrat用-1替代，并将这一列定义为标签列

In [210]:
vote_data = vote_data.replace('republican',1).replace('democrat',-1)
vote_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,-1,1,-1,1,1,1,-1,-1,-1,1,0,1,1,1,-1,1
1,1,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,0
2,-1,0,1,1,0,1,1,-1,-1,-1,-1,1,-1,1,1,-1,-1
3,-1,-1,1,1,-1,0,1,-1,-1,-1,-1,1,-1,1,-1,-1,1
4,-1,1,1,1,-1,1,1,-1,-1,-1,-1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,1,-1,-1,1,1,1,1,-1,-1,1,1,-1,1,1,1,-1,1
431,-1,-1,-1,1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,-1,1
432,1,-1,0,-1,1,1,1,-1,-1,-1,-1,1,1,1,1,-1,1
433,1,-1,-1,-1,1,1,1,0,0,0,0,-1,1,1,1,-1,1


交叉验证:划分数据的训练集与测试集

**train_test_split (*arrays，test_size, train_size, rondom_state=None, shuffle=True, stratify=None)**

    arrays：特征数据和标签数据（array，list，dataframe等类型），要求所有数据长度相同。
    test_size / train_size: 测试集/训练集的大小，若输入小数表示比例，若输入整数表示数据个数。
    rondom_state：随机种子（一个整数），其实就是一个划分标记，对于同一个数据集，如果rondom_state相同，则划分结果也相同。
    shuffle：是否打乱数据的顺序，再划分，默认True。
    stratify：none或者array/series类型的数据，表示按这列进行分层采样。

随机种子详解：

	其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。

下面我们对数据进行选择和测试，划分训练集、测试集和相关的标签集

采用train_test_split分割，其中的参数：test_size=0.2 表示测试集与训练集的比值是0.2

In [211]:
from sklearn.model_selection import train_test_split
label = vote_data[0]
sets = vote_data.iloc[:,1:].values
print(sets.shape)
train_x, test_x, train_y, test_y = train_test_split(sets, label, test_size=0.2, random_state=1)

(435, 16)


## knn

参看B1.classification.ipynb文件的介绍

## 数据集的验证

其中列表左边的一列为分类的标签名，右边support列为每个标签的出现次数．avg / total行为各列的均值（support列为总和）．
precision recall f1-score三列分别为各个类别的精确度/召回率及 F1值．

In [212]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

knn = KNeighborsClassifier()

knn = knn.fit(train_x, train_y)

answer_knn = knn.predict(test_x)
print("Prediction All Done!")
end =time.perf_counter()
print("knn程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for knn:')
print(classification_report(test_y, answer_knn))

Prediction All Done!
knn程序运行时间为： 0.10796850000042468 s
Generating reports......


The classification report for knn:
              precision    recall  f1-score   support

          -1       1.00      0.89      0.94        56
           1       0.84      1.00      0.91        31

    accuracy                           0.93        87
   macro avg       0.92      0.95      0.93        87
weighted avg       0.94      0.93      0.93        87



从报告来看，精确度达到90%以上，说明knn算法总体比较成功，一方面是数据；量足够大的体现，
一方面也是knn算法本身对于本模型比较适应。

下面尝试使用决策树

## 决策树

相关内容参见 B1.classification.ipynb 文件

这里使用from sklearn.tree import DecisionTreeClassifier实现分类器

In [213]:
from sklearn.tree import DecisionTreeClassifier # 决策树分类器
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

dtree = DecisionTreeClassifier()

dtree = dtree.fit(train_x, train_y)

answer_dtree = dtree.predict(test_x)
print("Prediction All Done!")
end =time.perf_counter()
print("决策树程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for dtree:')
print(classification_report(test_y, answer_dtree))

Prediction All Done!
决策树程序运行时间为： 0.09842220000064117 s
Generating reports......


The classification report for dtree:
              precision    recall  f1-score   support

          -1       0.96      0.96      0.96        56
           1       0.94      0.94      0.94        31

    accuracy                           0.95        87
   macro avg       0.95      0.95      0.95        87
weighted avg       0.95      0.95      0.95        87



可以看到比knn更高的精确度，但是对应的时间消耗明显大于knn算法，如果数据量更大，可能耗费时间更多

下面我们进行朴素贝叶斯模型计算

## 朴素贝叶斯

相关内容参见 B1.classification.ipynb 文件
分成高斯朴素贝叶斯、伯努利朴素贝叶斯等等，下面都去尝试一下

这里使用 from sklearn.naive_bayes import GaussianNB
实现分类器

In [214]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

gnb = GaussianNB()

gnb = gnb.fit(train_x, train_y)

answer_gnb = gnb.predict(test_x)
print("Prediction All Done!")
end =time.perf_counter()
print("高斯朴素贝叶斯程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for GaussianNB:')
print(classification_report(test_y, answer_gnb))

Prediction All Done!
高斯朴素贝叶斯程序运行时间为： 0.0044735000001310254 s
Generating reports......


The classification report for GaussianNB:
              precision    recall  f1-score   support

          -1       0.98      0.95      0.96        56
           1       0.91      0.97      0.94        31

    accuracy                           0.95        87
   macro avg       0.95      0.96      0.95        87
weighted avg       0.96      0.95      0.95        87



从得到的数据看，高斯朴素贝叶斯运算时间更短而且准确率相对较高，对该模型也比较合适

下面尝试伯努利模型朴素贝叶斯

In [215]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

gnb = BernoulliNB()

gnb = gnb.fit(train_x, train_y)

answer_gnb = gnb.predict(test_x)
print("Prediction All Done!")
end =time.perf_counter()
print("针对多元伯努利模型程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for BernoulliNB:')
print(classification_report(test_y, answer_gnb))


Prediction All Done!
针对多元伯努利模型程序运行时间为： 0.023406100000102015 s
Generating reports......


The classification report for BernoulliNB:
              precision    recall  f1-score   support

          -1       0.96      0.86      0.91        56
           1       0.78      0.94      0.85        31

    accuracy                           0.89        87
   macro avg       0.87      0.90      0.88        87
weighted avg       0.90      0.89      0.89        87



可以看到，让贝叶斯模型修改以后，对应的准确度反而下降了，说明本项目并不适合多元伯努利模型，而且后者时间耗费更高

那多项式模型呢？

In [216]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

gnb = MultinomialNB()

gnb = gnb.fit(train_x, train_y)

answer_gnb = gnb.predict(test_x)
print("Prediction All Done!")
end =time.perf_counter()
print("针对多项式模型程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for MultinomialNB:')
print(classification_report(test_y, answer_gnb))

ValueError: Negative values in data passed to MultinomialNB (input X)

多项式模型在本实验会报错，因为数据定义时存在负值,MultinomialNB规定不能存在负数。


### 综上，我们可以得出在本次选举数据集中，高斯朴素贝叶斯模型和决策树模型是最合适的

高斯贝叶斯属于比较特殊的一类分类器，其分类效果在二分数据上有比较高的优势

决策树模型在相对短的时间内能够对大型数据源做出可行且效果良好的结果。

## 2. 图片信息分类技术

数据集： data/segment-tain   data/segment-test  两个数据集，前者训练集，后者测试集

请注意数据的说明也在前面，因此提取数据的时候需要把上面的内容全部去掉！

因此我们需要对文本先提取数据

数据说明：

    区域质心列数值
    区域质心行数值
    区域像素计数数字
    短线密度-5数值
    短线密度-2数值
    vedge平均数值
    vegde sd数值
    对冲均值数值
    对冲sd数值
    强度平均数值
    RAWLED平均数值
    rawblue平均数值
    rawgreen平均数值
    exred平均数值
    exblue平均数值
    exgreen平均数值
    值平均数值
    饱和平均数值
    色调平均数值

In [217]:
with open('data/segment-tain.txt','r') as file:
    lists = file.readlines()
file.close()
Targetstring = '@data\n'
print(lists.index(Targetstring)+2)
pointer = lists.index(Targetstring)+2
lister  = lists[pointer:]
# print(lister)
with open('data/tain.csv','w') as wfile:
    wfile.writelines(lister)

107


同理，对data/segment-test.txt 做相同的操作,以data为标记对数据继续处理，
实现相关的操作

In [218]:
with open('data/segment-test.txt','r') as file:
    lists = file.readlines()
file.close()
Targetstring = '@data\n'
print(lists.index(Targetstring)+2)
pointer = lists.index(Targetstring)+2
lister  = lists[pointer:]
# print(lister)
with open('data/test.csv','w') as wfile:
    wfile.writelines(lister)

106


现在我们有了两个csv格式文件，需要对数据预处理

In [226]:
train = pd.read_csv('data/tain.csv',header=None)
print(train)
print(train.isnull().any())

       0    1   2         3         4         5          6         7   \
0      38  189   9  0.000000  0.000000  1.000000   0.222222  6.222220   
1      25  199   9  0.000000  0.000000  1.111110   0.607407  1.055560   
2      49  139   9  0.000000  0.000000  0.166667   0.077778  0.333333   
3      63  220   9  0.000000  0.000000  3.055560  15.263000  3.666670   
4     161  135   9  0.000000  0.000000  0.055556   0.136083  0.111111   
...   ...  ...  ..       ...       ...       ...        ...       ...   
1495  113  123   9  0.111111  0.000000  1.444440   0.620633  1.944440   
1496  245   93   9  0.111111  0.000000  2.888890   3.896310  1.944440   
1497  207  215   9  0.000000  0.000000  3.222220   2.040330  2.500000   
1498  242   65   9  0.000000  0.000000  0.555555   0.272166  1.000000   
1499  145  155   9  0.000000  0.111111  4.944440  35.618500  9.944440   

              8          9          10         11        12        13  \
0      33.318500  29.074100  26.333300   35.22220 

In [227]:
test = pd.read_csv('data/test.csv',header=None)
print(test)
print(test.isnull().any())

      0    1   2    3    4         5         6         7         8   \
0    144   35   9  0.0  0.0  2.333330  2.033060  2.055560  1.730980   
1    118  180   9  0.0  0.0  1.944440  1.481990  3.111110  1.088660   
2      6  174   9  0.0  0.0  1.888890  1.007410  2.888890  4.029630   
3    152  220   9  0.0  0.0  0.944445  0.685185  1.444440  2.162960   
4    189  142   9  0.0  0.0  0.000000  0.000000  0.000000  0.000000   
..   ...  ...  ..  ...  ...       ...       ...       ...       ...   
805  221  111   9  0.0  0.0  0.611111  0.240741  0.388889  0.240741   
806   44   79   9  0.0  0.0  0.444444  0.344265  0.777779  0.403686   
807  230   41   9  0.0  0.0  0.888891  0.688530  1.888890  1.241270   
808  217   77   9  0.0  0.0  1.555560  2.740740  1.666670  0.533333   
809   53  134   9  0.0  0.0  2.166670  0.300000  0.277778  0.018519   

            9          10        11        12        13        14        15  \
0     37.59260   32.33330   47.4444   33.0000 -15.77780  29.55560 -1

由于维度的类型并没有显示，我们这里直接设定header=None头部设空

接下来，考虑到存在string类型的文本，我们需要把这些文本进行进一步处理，用数字替代

观察到属性：class {brickface,sky,foliage,cement,window,path,grass}，
下面我们对数据进行处理加工

In [228]:
train = train.replace('brickface',1).replace('sky',2).replace('foliage',3).replace('cement',4).replace('window',5).replace('path',6).replace('grass',7)
test = test.replace('brickface',1).replace('sky',2).replace('foliage',3).replace('cement',4).replace('window',5).replace('path',6).replace('grass',7)
test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,144,35,9,0.0,0.0,2.333330,2.033060,2.05556,1.73098,37.5926,32.3333,47.4444,33.0000,-15.77780,29.55560,-13.7778,47.4444,0.319714,-2.13876,4
1,118,180,9,0.0,0.0,1.944440,1.481990,3.11111,1.08866,48.5556,44.1111,59.0000,42.5556,-13.33330,31.33330,-18.0000,59.0000,0.278822,-1.99604,6
2,6,174,9,0.0,0.0,1.888890,1.007410,2.88889,4.02963,19.0741,15.1111,17.7778,24.3333,-11.88890,-3.88889,15.7778,24.3333,0.381867,2.39502,7
3,152,220,9,0.0,0.0,0.944445,0.685185,1.44444,2.16296,14.6296,11.5556,13.1111,19.2222,-9.22222,-4.55556,13.7778,19.2222,0.416705,2.30688,7
4,189,142,9,0.0,0.0,0.000000,0.000000,0.00000,0.00000,0.0000,0.0000,0.0000,0.0000,0.00000,0.00000,0.0000,0.0000,0.000000,0.00000,5


可以看到明显数据变成了全部以数字表示的状态集，下一步，划分相关的标签集和其他集

可以看到相关的数据样例

In [242]:
label_train_data = train[19]

oth_train_data = train.iloc[:,:-1].values

print(label_train_data)

label_test_data = test[19]
oth_test_data =  test.iloc[:,:-1].values
print(oth_test_data)

0       6
1       7
2       3
3       7
4       5
       ..
1495    4
1496    2
1497    7
1498    5
1499    5
Name: 19, Length: 1500, dtype: int64
[[ 1.44000e+02  3.50000e+01  9.00000e+00 ...  4.74444e+01  3.19714e-01
  -2.13876e+00]
 [ 1.18000e+02  1.80000e+02  9.00000e+00 ...  5.90000e+01  2.78822e-01
  -1.99604e+00]
 [ 6.00000e+00  1.74000e+02  9.00000e+00 ...  2.43333e+01  3.81867e-01
   2.39502e+00]
 ...
 [ 2.30000e+02  4.10000e+01  9.00000e+00 ...  1.38889e+02  2.06392e-01
  -2.28080e+00]
 [ 2.17000e+02  7.70000e+01  9.00000e+00 ...  4.82222e+01  2.69192e-01
  -1.93207e+00]
 [ 5.30000e+01  1.34000e+02  9.00000e+00 ...  1.12222e+01  5.87234e-01
  -2.13846e+00]]


## knn

In [244]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

knn = KNeighborsClassifier()

knn = knn.fit(oth_train_data, label_train_data)

answer_knn2 = knn.predict(oth_test_data)
print("Prediction All Done!")
end =time.perf_counter()
print("knn程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for knn:')
print(classification_report(label_test_data, answer_knn2))

Prediction All Done!
knn程序运行时间为： 0.11474939999970957 s
Generating reports......


The classification report for knn:
              precision    recall  f1-score   support

           1       0.89      0.95      0.92       125
           2       1.00      1.00      1.00       110
           3       0.89      0.89      0.89       122
           4       0.92      0.89      0.90       110
           5       0.86      0.81      0.84       126
           6       0.97      1.00      0.98        94
           7       1.00      0.99      1.00       123

    accuracy                           0.93       810
   macro avg       0.93      0.93      0.93       810
weighted avg       0.93      0.93      0.93       810



## 决策树

In [24]:
from sklearn.tree import DecisionTreeClassifier # 决策树分类器
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

dtree = DecisionTreeClassifier()

dtree = dtree.fit(oth_train_data, label_train_data)

answer_dtree = dtree.predict(oth_test_data)
print("Prediction All Done!")
end =time.perf_counter()
print("决策树程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for dtree:')
print(classification_report(label_test_data, answer_dtree))

NameError: name 'oth_train_data' is not defined

决策树在第一次运行时往往比较慢，后面运行速度基本和knn一样快了，在本次实验中决策树的准确度也极高

## 贝叶斯·高斯

In [25]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

gnb = GaussianNB()

gnb = gnb.fit(oth_train_data, label_train_data)

answer_gnb = gnb.predict(oth_test_data)
print("Prediction All Done!")
end =time.perf_counter()
print("高斯朴素贝叶斯程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for GaussianNB:')
print(classification_report(label_test_data, answer_gnb))


NameError: name 'oth_train_data' is not defined

与上一个实验不同的是，这次的高斯朴素贝叶斯的准确率、召回率等等都比较低。朴素贝叶斯一般在小规模数据上的表现很好，适合进行多分类任务，
但是遇到数据量规模比较大的情况就不太适合了。

## 贝叶斯·伯努利模型

In [26]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report # 结果预测评估模块
import time

start =  time.perf_counter()

gnb = BernoulliNB()

gnb = gnb.fit(oth_train_data, label_train_data)

answer_gnb = gnb.predict(oth_test_data)
print("Prediction All Done!")
end =time.perf_counter()
print("针对多元伯努利模型程序运行时间为：",end-start,"s")
print("Generating reports......")
print('\n\nThe classification report for BernoulliNB:')
print(classification_report(label_test_data, answer_gnb))

NameError: name 'oth_train_data' is not defined

相关的效率更低了，表明贝叶斯不是本次图像分类实验的比较好的选择。

总结：本次图像分类的实验的更加适合使用knn模型和决策树模型